In [1]:
# %% Install necessary libraries
!pip install transformers accelerate trl peft bitsandbytes sympy matplotlib numpy scikit-learn datasets -q
!pip install unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 6.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 11.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

In [2]:
# %% Imports
import json
import torch
import sympy as sp
import os
import matplotlib
matplotlib.use('Agg')  # For headless environments
import matplotlib.pyplot as plt
import numpy as np
import re
import shutil
import glob
from sklearn.model_selection import train_test_split

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    GenerationConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding
)
from unsloth import FastLanguageModel
from trl import PPOTrainer, PPOConfig
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training


2025-05-18 17:28:47.533291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747589327.964289      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747589328.080580      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_35/2023442122.py:24: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# folder_to_delete = "/kaggle/working/sft_model"
# if os.path.exists(folder_to_delete):
#     shutil.rmtree(folder_to_delete)
#     print(f"Successfully deleted the folder: {folder_to_delete}")
# else:
#     print(f"The folder does not exist: {folder_to_delete}")

# %% Define directories
base_dir = "/kaggle/working/"
os.makedirs(base_dir, exist_ok=True)

output_dir = os.path.join(base_dir, "outputs")
plots_dir = os.path.join(output_dir, "plots")
checkpoint_dir = os.path.join(output_dir, "checkpoints")
base_model_dir = os.path.join(output_dir, "gpt2_xl_sft_rlhf_base")
reward_base_model_dir = os.path.join(output_dir, "qwen_reward_base")
reward_trained_model_dir = os.path.join(output_dir, "qwen_reward_trained")
sft_model_save_path = os.path.join(output_dir, "sft_model")
ppo_model_save_path = os.path.join(output_dir, "ppo_model")

for dir_path in [output_dir, plots_dir, checkpoint_dir, base_model_dir, reward_base_model_dir, reward_trained_model_dir, sft_model_save_path, ppo_model_save_path]:
    os.makedirs(dir_path, exist_ok=True)

# # %% Cleanup checkpoints directory
# if os.path.exists(checkpoint_dir):
#     shutil.rmtree(checkpoint_dir)
#     print(f"Deleted the directory: {checkpoint_dir}")
# else:
#     print(f"The directory {checkpoint_dir} does not exist.")

In [31]:
# # %% Cleanup checkpoints directory
# if os.path.exists(checkpoint_dir):
#     shutil.rmtree(checkpoint_dir)
#     print(f"Deleted the directory: {checkpoint_dir}")
# else:
#     print(f"The directory {checkpoint_dir} does not exist.")

Deleted the directory: /kaggle/working/outputs/checkpoints


In [33]:
# import os

# # Define the base directory and output directory
# base_dir = "/kaggle/working/"
# output_dir = os.path.join(base_dir, "outputs")
# checkpoint_dir = os.path.join(output_dir, "checkpoints")

# # Create the output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# # Check if the checkpoint directory exists
# if not os.path.exists(checkpoint_dir):
#     os.makedirs(checkpoint_dir)
#     print(f"Checkpoint directory created at: {checkpoint_dir}")
# else:
#     print(f"Checkpoint directory already exists at: {checkpoint_dir}")


Checkpoint directory created at: /kaggle/working/outputs/checkpoints


In [4]:
# %% Load and Prepare Datasets
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [5]:
sft_raw_data = load_jsonl("/kaggle/input/mini-pipeline-datasets/sft_raw_data.jsonl")
rlhf_raw_data = load_jsonl("/kaggle/input/mini-pipeline-datasets/reformatted_rlhf_data.jsonl")
reward_raw_data = load_jsonl("/kaggle/input/mini-pipeline-datasets/reformatted_reward_data.jsonl")

train_sft_data, val_sft_data = train_test_split(sft_raw_data, test_size=0.1, random_state=42)
sft_dataset = Dataset.from_list(train_sft_data)
val_dataset = Dataset.from_list(val_sft_data)

rlhf_dataset = Dataset.from_list(rlhf_raw_data)
train_rm_data, val_rm_data = train_test_split(reward_raw_data, test_size=0.1, random_state=42)
train_rm_dataset = Dataset.from_list(train_rm_data)
val_rm_dataset = Dataset.from_list(val_rm_data)
reward_val = Dataset.from_list(reward_raw_data)

In [6]:
# %% Helper function to extract boxed answer
def extract_boxed_answer(text):
    matches = re.findall(r"\\boxed{([^}]*)}", text)
    if matches:
        return matches[-1].strip()
    fallback = re.findall(r"[-+]?\d*\.\d+|\d+", text)
    return fallback[-1].strip() if fallback else text.strip()

In [9]:
# Remove Unsloth Cache
unsloth_cache = os.path.expanduser("~/.cache/huggingface/")
try:
    for f in os.listdir(unsloth_cache):
        if "unsloth" in f:
            path = os.path.join(unsloth_cache, f)
            if os.path.isfile(path):
                os.remove(path)
            else:
                shutil.rmtree(path)
except Exception as e:
    print(f"Cache cleanup warning: {e}")

In [10]:
import os
import torch
from unsloth import FastLanguageModel
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from transformers import GenerationConfig, AutoConfig

# Define base_model_dir
base_model_dir = "/kaggle/working/outputs/gpt2_xl_sft_rlhf_base"  # Adjust as needed

max_seq_length = 256  # Reduced for memory
dtype = torch.float16  # Use float16 for compatibility
load_in_4bit = False  # Explicitly disable 4-bit quantization

sft_rlhf_model_name = "gpt2-xl"

print(f"Loading SFT/RLHF Model: {sft_rlhf_model_name}")

try:
    if os.path.exists(base_model_dir) and os.path.exists(os.path.join(base_model_dir, "config.json")):
        print(f"Loading GPT-2 XL from {base_model_dir}")
        # Load configuration to verify
        model_config = AutoConfig.from_pretrained(base_model_dir)
        model, tokenizer = FastLanguageModel.from_pretrained(
            base_model_dir,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit
        )
    else:
        print(f"Downloading GPT-2 XL from Hugging Face and saving to {base_model_dir}")
        # Create directory if it doesn't exist
        os.makedirs(base_model_dir, exist_ok=True)
        model_config = AutoConfig.from_pretrained(sft_rlhf_model_name)
        model, tokenizer = FastLanguageModel.from_pretrained(
            sft_rlhf_model_name,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit
        )
        model.save_pretrained(base_model_dir)
        tokenizer.save_pretrained(base_model_dir)
        print(f"✅ Successfully downloaded and saved GPT-2 XL to {base_model_dir}")
        print(f"Contents of {base_model_dir}: {os.listdir(base_model_dir)}")

except TypeError as e:
    print(f"\n❌ Error during model loading: {e}")
    print("This often indicates a corrupted or incomplete model configuration.")
    print(f"Please try deleting the local model directory '{base_model_dir}' and run the script again to force a fresh download.")
    print("If the issue persists, consider using transformers.AutoModelForCausalLM directly as a fallback.")
    exit(1)  # Exit the script if model loading fails

except Exception as e:
    print(f"\n❌ Unexpected error during model loading: {e}")
    print(f"Please check network connectivity, disk space, or compatibility with {sft_rlhf_model_name}.")
    print(f"Try deleting '{base_model_dir}' and rerun.")
    exit(1)

# Move model to GPU
if torch.cuda.is_available():
    model = model.to("cuda:0")
    print(f"Model moved to GPU: {next(model.parameters()).device}")

# Set pad token for tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
    print(f"Tokenizer pad token set to: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")

# Prepare model for training
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

# Apply LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # Suitable for GPT-2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
FastLanguageModel.for_training(model)

# Set generation config with specific parameters
model.generation_config = GenerationConfig(
    max_length=max_seq_length,
    pad_token_id=model.config.pad_token_id,
    eos_token_id=model.config.eos_token_id
)

print("✅ SFT/RLHF Model is ready for fine-tuning with LoRA applied (no quantization).")

Loading SFT/RLHF Model: gpt2-xl
==((====))==  Unsloth 2025.5.6: Fast Gpt2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

gpt2-xl does not have a padding token! Will use pad_token = <|endoftext|>.
✅ Successfully downloaded and saved GPT-2 XL to /kaggle/working/outputs/gpt2_xl_sft_rlhf_base
Contents of /kaggle/working/outputs/gpt2_xl_sft_rlhf_base: ['tokenizer_config.json', 'generation_config.json', 'merges.txt', 'model.safetensors', 'vocab.json', 'config.json', 'special_tokens_map.json', 'tokenizer.json']
Model moved to GPU: cuda:0


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


✅ SFT/RLHF Model is ready for fine-tuning with LoRA applied (no quantization).


In [11]:
# %% Step 3: Supervised Fine-Tuning (SFT)
def tokenize_function_sft(examples):
    inputs = tokenizer(
        [f"Problem: {p}\nSolution: {s}" for p, s in zip(examples["problem"], examples["solution"])],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

sft_dataset = sft_dataset.map(tokenize_function_sft, batched=True)
val_dataset = val_dataset.map(tokenize_function_sft, batched=True)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [12]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def __init__(self, *args, processing_class=None, **kwargs):
        # Handle deprecated 'tokenizer' argument for compatibility
        if "tokenizer" in kwargs:
            processing_class = kwargs.pop("tokenizer")
        super().__init__(*args, processing_class=processing_class, **kwargs)
        self.train_losses = []
        self.val_accuracies = []
        self.val_reasonings = []

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Ignore num_items_in_batch since we don't need it for loss computation
        outputs = model(**inputs)
        loss = outputs.loss
        self.train_losses.append(loss.item())
        return (loss, outputs) if return_outputs else loss

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        accuracy, reasoning = evaluate_model(reward_val, self.model, self.tokenizer)
        self.val_accuracies.append(accuracy)
        self.val_reasonings.append(reasoning)
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        metrics[f"{metric_key_prefix}_accuracy"] = accuracy
        metrics[f"{metric_key_prefix}_reasoning"] = reasoning
        return metrics

In [13]:
# %% Find latest checkpoint
def find_latest_checkpoint(checkpoint_dir):
    checkpoint_paths = glob.glob(os.path.join(checkpoint_dir, "checkpoint-*"))
    return checkpoint_paths[-1] if checkpoint_paths else None

In [14]:
# %% SFT Training
from transformers import TrainingArguments
from peft import PeftModel

sft_args = TrainingArguments(
    output_dir=checkpoint_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=1,
    warmup_steps=10,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    fp16=True,
    weight_decay=0.01,
    report_to="none",
    save_total_limit=3,
    gradient_checkpointing=False
)

trainer = CustomTrainer(
    model=model,
    args=sft_args,
    train_dataset=sft_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer  # Use processing_class as fixed earlier
)

latest_checkpoint = find_latest_checkpoint(checkpoint_dir)
print("Starting SFT...")
trainer.train(resume_from_checkpoint=latest_checkpoint)

# Merge LoRA adapters and save the full model
print(f"Merging LoRA adapters and saving full SFT model to {sft_model_save_path}")
if isinstance(model, PeftModel):
    # Get the base model and merge the adapters
    model = model.merge_and_unload()  # Merges LoRA weights into the base model
else:
    print("Model is not a PeftModel; saving as-is.")

# Save the full model (now includes base weights and config)
model.save_pretrained(sft_model_save_path, torch_dtype=torch.float16)
tokenizer.save_pretrained(sft_model_save_path)
print("SFT model saved. Contents:", os.listdir(sft_model_save_path))

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 360 | Num Epochs = 1 | Total steps = 22
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 13,516,800/1,571,128,000 (0.86% trained)


Starting SFT...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


Merging LoRA adapters and saving full SFT model to /kaggle/working/outputs/sft_model
SFT model saved. Contents: ['model.safetensors.index.json', 'tokenizer_config.json', 'generation_config.json', 'merges.txt', 'model-00001-of-00002.safetensors', 'model-00002-of-00002.safetensors', 'vocab.json', 'config.json', 'special_tokens_map.json', 'tokenizer.json']


In [15]:
# %% Plot SFT Metrics
plt.figure(figsize=(10, 6))
plt.plot([i * sft_args.logging_steps for i in range(len(trainer.train_losses))], trainer.train_losses, label="Training Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("SFT Training Loss")
plt.legend()
plt.savefig(os.path.join(plots_dir, "sft_training_loss.png"))
plt.close()

plt.figure(figsize=(10, 6))
plt.plot([i * sft_args.eval_steps for i in range(len(trainer.val_accuracies))], trainer.val_accuracies, marker='o', label="Validation Accuracy")
plt.plot([i * sft_args.eval_steps for i in range(len(trainer.val_reasonings))], trainer.val_reasonings, marker='x', label="Validation Reasoning Quality")
plt.xlabel("Steps")
plt.ylabel("Score")
plt.title("SFT Validation Metrics")
plt.legend()
plt.savefig(os.path.join(plots_dir, "sft_validation_metrics.png"))
plt.close()

In [44]:
import torch
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Enable for debugging
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [16]:
# %% Step 4: Reward Model Training (with Disk Cleanup)
import os
import torch
import gc
import shutil
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

# Set PyTorch memory configuration to reduce fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check number of available GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")
for i in range(num_gpus):
    print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

# Check and display initial disk space
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Initial disk usage - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# Aggressive memory and disk cleanup
print("Performing aggressive cleanup before loading reward model...")
# Delete all known references to previous objects
global_vars_to_delete = [
    'model', 'trainer', 'sft_dataset', 'val_dataset', 'sft_args',
    'tokenized_train_rm_dataset', 'tokenized_val_rm_dataset'
]
for var in global_vars_to_delete:
    if var in globals():
        print(f"Deleting {var}...")
        del globals()[var]

# Delete GPT-2 XL base model if present (adjust path if different)
gpt2_base_dir = os.path.join("/kaggle/working/outputs", "gpt2-xl-base")  # Adjust path if needed
if os.path.exists(gpt2_base_dir):
    shutil.rmtree(gpt2_base_dir)
    print(f"Deleted GPT-2 XL base model from {gpt2_base_dir}")

# Delete old checkpoints to free space
checkpoint_dir = "/kaggle/working/outputs/checkpoints"
if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))]
    if len(checkpoints) > 1:
        for chk in checkpoints[:-1]:  # Keep the latest checkpoint
            shutil.rmtree(os.path.join(checkpoint_dir, chk))
            print(f"Deleted checkpoint: {chk}")

# GPU memory cleanup
torch.cuda.reset_peak_memory_stats()
for _ in range(3):
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.synchronize()

# Check memory and disk usage after cleanup
for i in range(num_gpus):
    print(f"GPU {i} memory allocated after cleanup: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved after cleanup: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk usage after cleanup - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

if any(torch.cuda.memory_allocated(i) / 1024**2 > 5000 for i in range(num_gpus)):
    print("⚠️ Warning: GPU memory usage is still high after cleanup.")
    print("Consider restarting the kernel and running only the reward model training step.")

print(f"\n⚡️ Starting Reward Model Training (Regression)\n")

reward_model_name_hub = "Qwen/Qwen2.5-1.5B-Instruct"
max_seq_length = 64

config_path = os.path.join(reward_base_model_dir, "config.json")
loaded_from_local = False

if os.path.exists(config_path):
    try:
        print(f"✅ Attempting to load Base Reward Model from {reward_base_model_dir}")
        reward_model = AutoModelForSequenceClassification.from_pretrained(
            reward_base_model_dir,
            num_labels=1,
            load_in_4bit=False,
            trust_remote_code=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
        reward_tokenizer = AutoTokenizer.from_pretrained(reward_base_model_dir, trust_remote_code=True)
        loaded_from_local = True
        print("✅ Successfully loaded Base Reward Model from local path.")
        print(f"Device map: {reward_model.hf_device_map if hasattr(reward_model, 'hf_device_map') else 'Not available'}")
    except Exception as e:
        print(f"❌ Failed to load Base Reward Model from {reward_base_model_dir}: {e}")

if not loaded_from_local:
    print(f"⏳ Base Reward Model not found locally or corrupted. Downloading from {reward_model_name_hub}")
    try:
        reward_model = AutoModelForSequenceClassification.from_pretrained(
            reward_model_name_hub,
            num_labels=1,
            load_in_4bit=False,
            trust_remote_code=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
        reward_tokenizer = AutoTokenizer.from_pretrained(reward_model_name_hub, trust_remote_code=True)
        print(f"💾 Saving downloaded Base Reward Model to {reward_base_model_dir}")
        reward_model.save_pretrained(reward_base_model_dir)
        reward_tokenizer.save_pretrained(reward_base_model_dir)
        print("✅ Successfully downloaded and saved Base Reward Model.")
        print(f"Device map: {reward_model.hf_device_map if hasattr(reward_model, 'hf_device_map') else 'Not available'}")
    except Exception as e:
        print(f"Fatal Error: Could not download or load Base Reward Model: {e}")
        raise e

reward_model = prepare_model_for_kbit_training(reward_model, use_gradient_checkpointing=True)

reward_lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

reward_model = get_peft_model(reward_model, reward_lora_config)

if reward_tokenizer.pad_token is None:
    reward_tokenizer.pad_token = reward_tokenizer.eos_token if reward_tokenizer.eos_token else "[PAD]"
    print(f"Reward Tokenizer pad token set to: {reward_tokenizer.pad_token}")

pad_token_id = reward_tokenizer.convert_tokens_to_ids(reward_tokenizer.pad_token)
if reward_model.config.pad_token_id is None or reward_model.config.pad_token_id == -1:
    reward_model.config.pad_token_id = pad_token_id
    print(f"Reward Model config pad_token_id set to: {reward_model.config.pad_token_id}")

def preprocess_reward_dataset_for_rm_regression(examples, tokenizer):
    texts = [f"Problem: {p}\nSolution: {s}" for p, s in zip(examples["problem"], examples["solution"])]
    labels = [float(l) for l in [item for sublist in examples["simulated_rewards"] for item in sublist]]
    tokenized_inputs = tokenizer(texts, truncation=True, padding="max_length", max_length=max_seq_length)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train_rm_dataset = train_rm_dataset.map(
    lambda x: preprocess_reward_dataset_for_rm_regression(x, reward_tokenizer),
    batched=True,
    remove_columns=[col for col in train_rm_dataset.column_names if col not in ['input_ids', 'attention_mask', 'labels']]
)
tokenized_val_rm_dataset = val_rm_dataset.map(
    lambda x: preprocess_reward_dataset_for_rm_regression(x, reward_tokenizer),
    batched=True,
    remove_columns=[col for col in val_rm_dataset.column_names if col not in ['input_ids', 'attention_mask', 'labels']]
)

# Memory and disk check after loading model
for i in range(num_gpus):
    print(f"GPU {i} memory allocated after loading reward model: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk usage before training - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

reward_args = TrainingArguments(
    output_dir=checkpoint_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    weight_decay=0.001,
    report_to="none",
    save_total_limit=1,  # Limit to 1 checkpoint to save disk space
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    dataloader_num_workers=2,
    dataloader_pin_memory=True
)

reward_trainer = Trainer(
    model=reward_model,
    tokenizer=reward_tokenizer,
    args=reward_args,
    train_dataset=tokenized_train_rm_dataset,
    eval_dataset=tokenized_val_rm_dataset,
    data_collator=DataCollatorWithPadding(reward_tokenizer, padding="max_length", max_length=max_seq_length)
)

print("Starting Reward Model Training...")
reward_trainer.train()
print(f"✅ Reward Model Training Completed and Saved to {reward_trained_model_dir}")

# Check disk space before saving the final model
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk usage before final save - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# Merge LoRA adapters and save the full model
print(f"Merging LoRA adapters and saving full reward model to {reward_trained_model_dir}")
if isinstance(reward_model, PeftModel):
    reward_model = reward_model.merge_and_unload()
else:
    print("Reward model is not a PeftModel; saving as-is.")
reward_model.save_pretrained(reward_trained_model_dir, torch_dtype=torch.float16)
reward_tokenizer.save_pretrained(reward_trained_model_dir)
print("Reward model saved. Contents:", os.listdir(reward_trained_model_dir))

# Clear GPU memory after training
print("Clearing GPU memory after reward model training...")
torch.cuda.empty_cache()
del reward_model
del reward_trainer
del tokenized_train_rm_dataset
del tokenized_val_rm_dataset
gc.collect()
for i in range(num_gpus):
    print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

Number of available GPUs: 2
GPU 0 memory allocated: 6228.20 MB
GPU 0 memory reserved: 7298.00 MB
GPU 1 memory allocated: 2.00 MB
GPU 1 memory reserved: 20.00 MB
Initial disk usage - Total: 19.52 GB, Used: 8.87 GB, Free: 10.63 GB
Performing aggressive cleanup before loading reward model...
Deleting model...
Deleting trainer...
Deleting sft_dataset...
Deleting val_dataset...
Deleting sft_args...
GPU 0 memory allocated after cleanup: 19.38 MB
GPU 0 memory reserved after cleanup: 2998.00 MB
GPU 1 memory allocated after cleanup: 2.00 MB
GPU 1 memory reserved after cleanup: 20.00 MB
Disk usage after cleanup - Total: 19.52 GB, Used: 8.87 GB, Free: 10.63 GB

⚡️ Starting Reward Model Training (Regression)

⏳ Base Reward Model not found locally or corrupted. Downloading from Qwen/Qwen2.5-1.5B-Instruct


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-1.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

...

GPU 0 memory allocated after loading reward model: 2902.34 MB
GPU 0 memory reserved: 3136.00 MB
GPU 1 memory allocated after loading reward model: 3079.62 MB
GPU 1 memory reserved: 3420.00 MB
Disk usage before training - Total: 19.52 GB, Used: 11.76 GB, Free: 7.74 GB
Starting Reward Model Training...


/tmp/ipykernel_35/1341984294.py:191: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  reward_trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 191 | Num Epochs = 1 | Total steps = 191
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 18,466,304/1,562,182,144 (1.18% trained)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling p

Step,Training Loss,Validation Loss
50,4.488800,4.251545
100,8.300800,3.955332
150,1.717100,3.818267


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

✅ Reward Model Training Completed and Saved to /kaggle/working/outputs/qwen_reward_trained
Disk usage before final save - Total: 19.52 GB, Used: 11.83 GB, Free: 7.68 GB
Merging LoRA adapters and saving full reward model to /kaggle/working/outputs/qwen_reward_trained
Reward model saved. Contents: ['model.safetensors.index.json', 'tokenizer_config.json', 'merges.txt', 'model-00001-of-00002.safetensors', 'model-00002-of-00002.safetensors', 'vocab.json', 'config.json', 'special_tokens_map.json', 'tokenizer.json', 'added_tokens.json']
Clearing GPU memory after reward model training...
GPU 0 memory allocated: 19.38 MB
GPU 0 memory reserved: 3208.00 MB
GPU 1 memory allocated: 18.38 MB
GPU 1 memory reserved: 3556.00 MB


In [28]:
import os
import torch
import gc
import shutil
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification
)
from peft import PeftModel
from trl import PPOTrainer, PPOConfig
from datasets import load_dataset

# Set PyTorch memory configuration to reduce fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check number of available GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")
for i in range(num_gpus):
    print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

# Check and display initial disk space
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Initial disk usage - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# Aggressive memory and disk cleanup
print("Performing aggressive cleanup before starting RLHF...")
# Delete all known references to previous objects
global_vars_to_delete = [
    'model', 'trainer', 'sft_dataset', 'val_dataset', 'sft_args',
    'tokenized_train_rm_dataset', 'tokenized_val_rm_dataset',
    'reward_model', 'reward_trainer'
]
for var in global_vars_to_delete:
    if var in globals():
        print(f"Deleting {var}...")
        del globals()[var]

# Delete the base Reward Model to free space
reward_base_model_dir = "/kaggle/working/outputs/qwen_reward_base"
if os.path.exists(reward_base_model_dir):
    shutil.rmtree(reward_base_model_dir)
    print(f"Deleted base Reward Model from {reward_base_model_dir}")

# Delete old checkpoints to free space
checkpoint_dir = "/kaggle/working/outputs/checkpoints"
if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))]
    if len(checkpoints) > 1:
        for chk in checkpoints[:-1]:  # Keep the latest checkpoint
            shutil.rmtree(os.path.join(checkpoint_dir, chk))
            print(f"Deleted checkpoint: {chk}")

# GPU memory cleanup
torch.cuda.reset_peak_memory_stats()
for _ in range(3):
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.synchronize()

# Check memory and disk usage after cleanup
for i in range(num_gpus):
    print(f"GPU {i} memory allocated after cleanup: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved after cleanup: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk usage after cleanup - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

print(f"\n⚡️ Starting Reinforcement Learning Fine-Tuning (RLHF)\n")

# Determine the device to load models on
# By explicitly setting a single device, we avoid potential cross-GPU tensor issues.
# Ensure the chosen GPU has enough VRAM for the entire model (approx. 6GB in float16 for your setup).
target_device = "cuda:0" # You can change this to "cuda:1" if GPU 1 is preferred or has more space

# Load SFT-trained model
sft_model_save_path = "/kaggle/working/outputs/sft_model"
print(f"Loading SFT-trained model in float16 from {sft_model_save_path}")
model = AutoModelForCausalLM.from_pretrained(
    sft_model_save_path,
    torch_dtype=torch.float16,
    device_map=target_device,  # FIX: Explicitly place on one GPU
    attn_implementation="eager"
)
if os.path.exists(os.path.join(sft_model_save_path, "adapter_model.safetensors")):
    # If adapter exists, ensure it's also loaded onto the same device
    model = PeftModel.from_pretrained(model, sft_model_save_path, torch_dtype=torch.float16, device_map=target_device) # FIX: Explicitly place on one GPU
for name, param in model.named_parameters():
    if "lora" in name.lower():
        param.data = param.data.to(torch.float16)
        print(f"Cast {name} to float16")
tokenizer = AutoTokenizer.from_pretrained(sft_model_save_path)
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None or tokenizer.pad_token == tokenizer.eos_token:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.convert_tokens_to_ids('[PAD]')
model.config.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
print(f"Model config - pad_token_id: {model.config.pad_token_id}, eos_token_id: {model.config.eos_token_id}")

# Load the trained Reward Model
reward_trained_model_dir = "/kaggle/working/outputs/qwen_reward_trained"
print(f"Loading trained Reward Model from {reward_trained_model_dir}")
reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_trained_model_dir,
    num_labels=1,
    torch_dtype=torch.float16,
    device_map=target_device,  # FIX: Explicitly place on one GPU
    trust_remote_code=True
)
reward_tokenizer = AutoTokenizer.from_pretrained(reward_trained_model_dir, trust_remote_code=True)

# Memory and disk check after loading models
for i in range(num_gpus):
    print(f"GPU {i} memory allocated after loading models: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk usage after loading models - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# Mock train_rm_dataset if it's not defined
# This part is crucial for the script to be runnable standalone for testing purposes.
if 'train_rm_dataset' not in globals():
    print("train_rm_dataset not found, creating a mock dataset for demonstration.")
    from datasets import Dataset
    # Create a dummy dataset with the expected 'problem' column
    dummy_data = {
        "problem": [
            "What is 2+2?",
            "Solve for x: x - 5 = 10",
            "What is the capital of France?",
            "Calculate the area of a rectangle with length 5 and width 3."
        ]
    }
    train_rm_dataset = Dataset.from_dict(dummy_data)
    print(f"Mock train_rm_dataset created with {len(train_rm_dataset)} examples.")

# Prepare dataset for RLHF
max_seq_length = 64
def preprocess_rlhf_dataset(examples, tokenizer):
    prompts = [f"Problem: {p}\nSolution: " for p in examples["problem"]]
    tokenized_inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt")
    return {"input_ids": tokenized_inputs["input_ids"], "attention_mask": tokenized_inputs["attention_mask"]}

rlhf_dataset = train_rm_dataset.map(
    lambda x: preprocess_rlhf_dataset(x, tokenizer),
    batched=True,
    remove_columns=[col for col in train_rm_dataset.column_names if col not in ['input_ids', 'attention_mask']]
)

# Configure PPO for RLHF
ppo_config = PPOConfig(
    batch_size=1,
    learning_rate=1e-5,
    num_ppo_epochs=1,
    mini_batch_size=1,
    gradient_accumulation_steps=1,
    #log_with=None
    ref_model_backend="cpu", # FIX: Offload the reference model to CPU
)

# Initialize PPO Trainer
ppo_trainer = PPOTrainer(
    args=ppo_config,
    model=model,
    ref_model=None,         # trl will manage the reference model based on ppo_config.ref_model_backend
    tokenizer=tokenizer,
    train_dataset=rlhf_dataset,
    reward_model=reward_model,
    value_model=model,      # Pass the 'model' itself as the value model
    data_collator=DataCollatorWithPadding(tokenizer, padding="max_length", max_length=max_seq_length)
)
print("PPOTrainer initialized.")

# RLHF Training Loop
print("Starting RLHF training with PPO...")
for step, batch in enumerate(ppo_trainer.dataloader):
    # Ensure input tensors are on the same device as the model
    query_tensors = batch["input_ids"].to(target_device)
    attention_mask = batch["attention_mask"].to(target_device)

    # Generate responses
    response_tensors = ppo_trainer.model.policy.generate(
        query_tensors,
        attention_mask=attention_mask,
        max_new_tokens=64, # Use max_new_tokens to specify how many NEW tokens to generate.
                           # Be aware: The total generated sequence length (input_length + max_new_tokens)
                           # might exceed the max_seq_length (64) used by reward_tokenizer,
                           # leading to truncation for reward calculation.
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1
    )

    # Prepare inputs for the Reward Model
    queries = [tokenizer.decode(q, skip_special_tokens=True) for q in query_tensors]
    responses = [tokenizer.decode(r, skip_special_tokens=True) for r in response_tensors]
    
    # Extract the problem part from each query
    reward_inputs = []
    for q, r in zip(queries, responses):
        # Split the query to isolate the problem part
        problem_part = q.split('Problem: ')[1].split('\nSolution: ')[0] if 'Problem: ' in q and '\nSolution: ' in q else q
        
        # Construct the reward input in the required format
        reward_input = f"Problem: {problem_part}\nSolution: {r}"
        reward_inputs.append(reward_input)

    # Tokenize for reward model. Ensure inputs are on the same device as the reward model.
    reward_tokenized = reward_tokenizer(reward_inputs, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt").to(target_device)

    # Get rewards
    with torch.no_grad():
        reward_outputs = reward_model(**reward_tokenized)
        rewards = reward_outputs.logits.squeeze()
        if rewards.ndim == 0: # Ensure it's a 1-dim tensor if batch_size=1
            rewards = rewards.unsqueeze(0)

    # PPO step
    stats = ppo_trainer.step(
        query_tensors,
        response_tensors,
        rewards
    )

    if step % 10 == 0:
        print(f"Step {step} - PPO stats: {stats}")

    if step >= 50:  # Limit for demonstration; adjust as needed
        print(f"Reached step {step}, stopping training.")
        break

# Save the RLHF-trained model
rlhf_model_save_path = "/kaggle/working/outputs/rlhf_model"
print(f"Saving RLHF-trained model to {rlhf_model_save_path}")
model.save_pretrained(rlhf_model_save_path, torch_dtype=torch.float16)
tokenizer.save_pretrained(rlhf_model_save_path)
print("RLHF model saved. Contents:", os.listdir(rlhf_model_save_path))

# Clear GPU memory after training
print("Clearing GPU memory after RLHF training...")
torch.cuda.empty_cache()
del model
del reward_model
del ppo_trainer
del rlhf_dataset
gc.collect()
for i in range(num_gpus):
    print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

# Final disk usage check
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Final disk usage - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

Number of available GPUs: 2
GPU 0 memory allocated: 14675.27 MB
GPU 0 memory reserved: 14892.00 MB
GPU 1 memory allocated: 19.16 MB
GPU 1 memory reserved: 1562.00 MB
Initial disk usage - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB
Performing aggressive cleanup before starting RLHF...
Deleting model...
Deleting reward_model...
GPU 0 memory allocated after cleanup: 14620.95 MB
GPU 0 memory reserved after cleanup: 14852.00 MB
GPU 1 memory allocated after cleanup: 18.38 MB
GPU 1 memory reserved after cleanup: 1558.00 MB
Disk usage after cleanup - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB

⚡️ Starting Reinforcement Learning Fine-Tuning (RLHF)

Loading SFT-trained model in float16 from /kaggle/working/outputs/sft_model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 40.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 3827 has 14.74 GiB memory in use. Of the allocated memory 14.30 GiB is allocated by PyTorch, and 255.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
!pip install --upgrade trl transformers accelerate bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached trl-0.17.0-py3-none-any.whl.metadata (12 kB)
Using cached trl-0.17.0-py3-none-any.whl (348 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 6.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.7 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.9.2,!=0.9.3,<=0.15.2,>=0.7.9, but you have trl 0.17.0 which is incompatible.
unsloth 2025.5.6 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.9.2,!=0.9.3,<=0.15.2,>=0.7.9, but you have trl 0.17.0 which is incompatible.


In [31]:
import os
import torch
import gc
import shutil
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification
)
from peft import PeftModel
from trl import PPOTrainer, PPOConfig
from datasets import load_dataset

# Set PyTorch memory configuration to reduce fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check number of available GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")
for i in range(num_gpus):
    print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

# Check and display initial disk space
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Initial disk usage - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# Aggressive memory and disk cleanup
print("Performing aggressive cleanup before starting RLHF...")
# Delete all known references to previous objects
global_vars_to_delete = [
    'model', 'trainer', 'sft_dataset', 'val_dataset', 'sft_args',
    'tokenized_train_rm_dataset', 'tokenized_val_rm_dataset',
    'reward_model', 'reward_trainer'
]
for var in global_vars_to_delete:
    if var in globals():
        print(f"Deleting {var}...")
        del globals()[var]

# Delete the base Reward Model to free space
reward_base_model_dir = "/kaggle/working/outputs/qwen_reward_base"
if os.path.exists(reward_base_model_dir):
    shutil.rmtree(reward_base_model_dir)
    print(f"Deleted base Reward Model from {reward_base_model_dir}")

# Delete old checkpoints to free space
checkpoint_dir = "/kaggle/working/outputs/checkpoints"
if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))]
    if len(checkpoints) > 1:
        for chk in checkpoints[:-1]:  # Keep the latest checkpoint
            shutil.rmtree(os.path.join(checkpoint_dir, chk))
            print(f"Deleted checkpoint: {chk}")

# GPU memory cleanup
torch.cuda.reset_peak_memory_stats()
for _ in range(3):
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.synchronize()

# Check memory and disk usage after cleanup
for i in range(num_gpus):
    print(f"GPU {i} memory allocated after cleanup: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk usage after cleanup - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

print(f"\n⚡️ Starting Reinforcement Learning Fine-Tuning (RLHF)\n")

# Determine the device to load models on
target_device = "cuda:0"

# Load SFT-trained model with 4-bit quantization
sft_model_save_path = "/kaggle/working/outputs/sft_model"
print(f"Loading SFT-trained model in 4-bit from {sft_model_save_path}")
model = AutoModelForCausalLM.from_pretrained(
    sft_model_save_path,
    load_in_4bit=True,
    device_map=target_device,
    attn_implementation="eager"
)
if os.path.exists(os.path.join(sft_model_save_path, "adapter_model.safetensors")):
    model = PeftModel.from_pretrained(model, sft_model_save_path, device_map=target_device)
for name, param in model.named_parameters():
    if "lora" in name.lower():
        param.data = param.data.to(torch.float16)
        print(f"Cast {name} to float16")
tokenizer = AutoTokenizer.from_pretrained(sft_model_save_path)
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None or tokenizer.pad_token == tokenizer.eos_token:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.convert_tokens_to_ids('[PAD]')
model.config.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
print(f"Model config - pad_token_id: {model.config.pad_token_id}, eos_token_id: {model.config.eos_token_id}")

# Load the trained Reward Model with 4-bit quantization
reward_trained_model_dir = "/kaggle/working/outputs/qwen_reward_trained"
print(f"Loading trained Reward Model in 4-bit from {reward_trained_model_dir}")
reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_trained_model_dir,
    num_labels=1,
    load_in_4bit=True,
    device_map=target_device,
    trust_remote_code=True
)
reward_tokenizer = AutoTokenizer.from_pretrained(reward_trained_model_dir, trust_remote_code=True)

# --- START OF MANUAL REF_MODEL CREATION ---
print("Creating reference model on CPU...")
# Load the base model again specifically for the reference model, directly onto CPU
# Ensure it's loaded in 4-bit for consistency if your main model is also 4-bit
ref_model = AutoModelForCausalLM.from_pretrained(
    sft_model_save_path,
    load_in_4bit=True, # Ensure consistency with main model if it's 4-bit
    device_map="cpu",  # <-- Load directly to CPU
    attn_implementation="eager"
)
# If the original SFT model was a PeftModel, the ref_model should also be a PeftModel
if os.path.exists(os.path.join(sft_model_save_path, "adapter_model.safetensors")):
    ref_model = PeftModel.from_pretrained(ref_model, sft_model_save_path, device_map="cpu")
# Ensure LoRA parameters are cast to float16 if they were trained as such
for name, param in ref_model.named_parameters():
    if "lora" in name.lower():
        param.data = param.data.to(torch.float16)

# --- END OF MANUAL REF_MODEL CREATION ---


# Memory and disk check after loading models
for i in range(num_gpus):
    print(f"GPU {i} memory allocated after loading models: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk usage after loading models - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# Mock train_rm_dataset if it's not defined
if 'train_rm_dataset' not in globals():
    print("train_rm_dataset not found, creating a mock dataset for demonstration.")
    from datasets import Dataset
    dummy_data = {
        "problem": [
            "What is 2+2?",
            "Solve for x: x - 5 = 10",
            "What is the capital of France?",
            "Calculate the area of a rectangle with length 5 and width 3."
        ]
    }
    train_rm_dataset = Dataset.from_dict(dummy_data)
    print(f"Mock train_rm_dataset created with {len(train_rm_dataset)} examples.")

# Prepare dataset for RLHF
max_seq_length = 64
def preprocess_rlhf_dataset(examples, tokenizer):
    prompts = [f"Problem: {p}\nSolution: " for p in examples["problem"]]
    tokenized_inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt")
    return {"input_ids": tokenized_inputs["input_ids"], "attention_mask": tokenized_inputs["attention_mask"]}

rlhf_dataset = train_rm_dataset.map(
    lambda x: preprocess_rlhf_dataset(x, tokenizer),
    batched=True,
    remove_columns=[col for col in train_rm_dataset.column_names if col not in ['input_ids', 'attention_mask']]
)

# Configure PPO for RLHF - NO ref_model_backend here
ppo_config = PPOConfig(
    batch_size=1,
    learning_rate=1e-5,
    num_ppo_epochs=1,
    mini_batch_size=1,
    gradient_accumulation_steps=1,
    #log_with=None # Remove this if not needed or causing issues
    # ref_model_backend="cpu", # <-- REMOVE THIS LINE
)

# Initialize PPO Trainer
ppo_trainer = PPOTrainer(
    args=ppo_config,
    model=model,
    ref_model=ref_model, # <-- Pass the manually created CPU reference model
    tokenizer=tokenizer,
    train_dataset=rlhf_dataset,
    reward_model=reward_model,
    value_model=model,
    data_collator=DataCollatorWithPadding(tokenizer, padding="max_length", max_length=max_seq_length)
)
print("PPOTrainer initialized.")

# RLHF Training Loop
print("Starting RLHF training with PPO...")
for step, batch in enumerate(ppo_trainer.dataloader):
    # Ensure input tensors are on the same device as the model
    query_tensors = batch["input_ids"].to(target_device)
    attention_mask = batch["attention_mask"].to(target_device)

    # Generate responses
    response_tensors = ppo_trainer.model.policy.generate(
        query_tensors,
        attention_mask=attention_mask,
        max_new_tokens=64,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1
    )

    # Prepare inputs for the Reward Model
    queries = [tokenizer.decode(q, skip_special_tokens=True) for q in query_tensors]
    responses = [tokenizer.decode(r, skip_special_tokens=True) for r in response_tensors]
    
    # Extract the problem part from each query
    reward_inputs = []
    for q, r in zip(queries, responses):
        problem_part = q.split('Problem: ')[1].split('\nSolution: ')[0] if 'Problem: ' in q and '\nSolution: ' in q else q
        reward_input = f"Problem: {problem_part}\nSolution: {r}"
        reward_inputs.append(reward_input)

    # Tokenize for reward model. Ensure inputs are on the same device as the reward model.
    reward_tokenized = reward_tokenizer(reward_inputs, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt").to(target_device)

    # Get rewards
    with torch.no_grad():
        reward_outputs = reward_model(**reward_tokenized)
        rewards = reward_outputs.logits.squeeze()
        if rewards.ndim == 0:
            rewards = rewards.unsqueeze(0)

    # PPO step
    stats = ppo_trainer.step(
        query_tensors,
        response_tensors,
        rewards
    )

    if step % 10 == 0:
        print(f"Step {step} - PPO stats: {stats}")

    if step >= 50:
        print(f"Reached step {step}, stopping training.")
        break

# Save the RLHF-trained model
rlhf_model_save_path = "/kaggle/working/outputs/rlhf_model"
print(f"Saving RLHF-trained model to {rlhf_model_save_path}")
model.save_pretrained(rlhf_model_save_path, torch_dtype=torch.float16)
tokenizer.save_pretrained(rlhf_model_save_path)
print("RLHF model saved. Contents:", os.listdir(rlhf_model_save_path))

# Clear GPU memory after training
print("Clearing GPU memory after RLHF training...")
torch.cuda.empty_cache()
del model
del reward_model
del ref_model # Also delete the reference model
del ppo_trainer
del rlhf_dataset
gc.collect()
for i in range(num_gpus):
    print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
    print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

# Final disk usage check
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Final disk usage - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

Number of available GPUs: 2
GPU 0 memory allocated: 8228.23 MB
GPU 0 memory reserved: 11100.00 MB
GPU 1 memory allocated: 18.38 MB
GPU 1 memory reserved: 1558.00 MB
Initial disk usage - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB
Performing aggressive cleanup before starting RLHF...
Deleting model...
Deleting reward_model...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


GPU 0 memory allocated after cleanup: 6021.14 MB
GPU 0 memory reserved: 9288.00 MB
GPU 1 memory allocated after cleanup: 18.38 MB
GPU 1 memory reserved: 1558.00 MB
Disk usage after cleanup - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB

⚡️ Starting Reinforcement Learning Fine-Tuning (RLHF)

Loading SFT-trained model in 4-bit from /kaggle/working/outputs/sft_model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

GPU 0 memory allocated after loading models: 8208.37 MB
GPU 0 memory reserved: 11056.00 MB
GPU 1 memory allocated after loading models: 18.38 MB
GPU 1 memory reserved: 1558.00 MB
Disk usage after loading models - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB


Map:   0%|          | 0/191 [00:00<?, ? examples/s]

PPOTrainer initialized.
Starting RLHF training with PPO...


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


AttributeError: 'UnslothPPOTrainer' object has no attribute 'step'

In [26]:
# # %% Step 5: Reinforcement Learning Fine-Tuning (RLHF)
# import os
# import torch
# import gc
# import shutil
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     DataCollatorWithPadding,
#     AutoModelForSequenceClassification
# )
# from peft import PeftModel
# from trl import PPOTrainer, PPOConfig
# from datasets import load_dataset

# # Set PyTorch memory configuration to reduce fragmentation
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# # Check number of available GPUs
# num_gpus = torch.cuda.device_count()
# print(f"Number of available GPUs: {num_gpus}")
# for i in range(num_gpus):
#     print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
#     print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

# # Check and display initial disk space
# total, used, free = shutil.disk_usage("/kaggle/working")
# print(f"Initial disk usage - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# # Aggressive memory and disk cleanup
# print("Performing aggressive cleanup before starting RLHF...")
# # Delete all known references to previous objects
# global_vars_to_delete = [
#     'model', 'trainer', 'sft_dataset', 'val_dataset', 'sft_args',
#     'tokenized_train_rm_dataset', 'tokenized_val_rm_dataset',
#     'reward_model', 'reward_trainer'
# ]
# for var in global_vars_to_delete:
#     if var in globals():
#         print(f"Deleting {var}...")
#         del globals()[var]

# # Delete the base Reward Model to free space
# reward_base_model_dir = "/kaggle/working/outputs/qwen_reward_base"
# if os.path.exists(reward_base_model_dir):
#     shutil.rmtree(reward_base_model_dir)
#     print(f"Deleted base Reward Model from {reward_base_model_dir}")

# # Delete old checkpoints to free space
# checkpoint_dir = "/kaggle/working/outputs/checkpoints"
# if os.path.exists(checkpoint_dir):
#     checkpoints = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))]
#     if len(checkpoints) > 1:
#         for chk in checkpoints[:-1]:  # Keep the latest checkpoint
#             shutil.rmtree(os.path.join(checkpoint_dir, chk))
#             print(f"Deleted checkpoint: {chk}")

# # GPU memory cleanup
# torch.cuda.reset_peak_memory_stats()
# for _ in range(3):
#     torch.cuda.empty_cache()
#     gc.collect()
#     torch.cuda.synchronize()

# # Check memory and disk usage after cleanup
# for i in range(num_gpus):
#     print(f"GPU {i} memory allocated after cleanup: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
#     print(f"GPU {i} memory reserved after cleanup: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
# total, used, free = shutil.disk_usage("/kaggle/working")
# print(f"Disk usage after cleanup - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# print(f"\n⚡️ Starting Reinforcement Learning Fine-Tuning (RLHF)\n")

# # Load SFT-trained model
# sft_model_save_path = "/kaggle/working/outputs/sft_model"
# print(f"Loading SFT-trained model in float16 from {sft_model_save_path}")
# model = AutoModelForCausalLM.from_pretrained(
#     sft_model_save_path,
#     torch_dtype=torch.float16,
#     device_map="auto",  # Distribute across GPUs
#     attn_implementation="eager"
# )
# if os.path.exists(os.path.join(sft_model_save_path, "adapter_model.safetensors")):
#     model = PeftModel.from_pretrained(model, sft_model_save_path, torch_dtype=torch.float16, device_map="auto")
# for name, param in model.named_parameters():
#     if "lora" in name.lower():
#         param.data = param.data.to(torch.float16)
#         print(f"Cast {name} to float16")
# tokenizer = AutoTokenizer.from_pretrained(sft_model_save_path)
# tokenizer.padding_side = 'left'
# if tokenizer.pad_token is None or tokenizer.pad_token == tokenizer.eos_token:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))
# model.config.pad_token_id = tokenizer.convert_tokens_to_ids('[PAD]')
# model.config.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
# print(f"Model config - pad_token_id: {model.config.pad_token_id}, eos_token_id: {model.config.eos_token_id}")

# # Load the trained Reward Model
# reward_trained_model_dir = "/kaggle/working/outputs/qwen_reward_trained"
# print(f"Loading trained Reward Model from {reward_trained_model_dir}")
# reward_model = AutoModelForSequenceClassification.from_pretrained(
#     reward_trained_model_dir,
#     num_labels=1,
#     torch_dtype=torch.float16,
#     device_map="auto",  # Distribute across GPUs
#     trust_remote_code=True
# )
# reward_tokenizer = AutoTokenizer.from_pretrained(reward_trained_model_dir, trust_remote_code=True)

# # Memory and disk check after loading models
# for i in range(num_gpus):
#     print(f"GPU {i} memory allocated after loading models: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
#     print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
# total, used, free = shutil.disk_usage("/kaggle/working")
# print(f"Disk usage after loading models - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

# # Prepare dataset for RLHF
# max_seq_length = 64
# def preprocess_rlhf_dataset(examples, tokenizer):
#     prompts = [f"Problem: {p}\nSolution: " for p in examples["problem"]]
#     tokenized_inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt")
#     return {"input_ids": tokenized_inputs["input_ids"], "attention_mask": tokenized_inputs["attention_mask"]}

# rlhf_dataset = train_rm_dataset.map(
#     lambda x: preprocess_rlhf_dataset(x, tokenizer),
#     batched=True,
#     remove_columns=[col for col in train_rm_dataset.column_names if col not in ['input_ids', 'attention_mask']]
# )

# # Configure PPO for RLHF
# ppo_config = PPOConfig(
#     batch_size=1,
#     learning_rate=1e-5,
#     num_ppo_epochs=1,
#     mini_batch_size=1,
#     gradient_accumulation_steps=1,
#     #log_with=None
# )

# # # Initialize PPO Trainer
# # ppo_trainer = PPOTrainer(
# #     config=ppo_config,
# #     model=model,
# #     ref_model=None,  # Use the SFT model as reference (no separate ref model needed)
# #     tokenizer=tokenizer,
# #     dataset=rlhf_dataset,
# #     data_collator=DataCollatorWithPadding(tokenizer, padding="max_length", max_length=max_seq_length)
# # )

# # Initialize PPO Trainer
# ppo_trainer = PPOTrainer(
#     args=ppo_config,
#     model=model,
#     ref_model=None,         # Use the SFT model as reference
#     tokenizer=tokenizer,
#     train_dataset=rlhf_dataset,
#     reward_model=reward_model,
#     value_model=model,      # FIX: Pass the 'model' itself as the value model
#     data_collator=DataCollatorWithPadding(tokenizer, padding="max_length", max_length=max_seq_length)
# )
# print("PPOTrainer initialized.")

# # RLHF Training Loop
# print("Starting RLHF training with PPO...")
# for step, batch in enumerate(ppo_trainer.dataloader):
#     query_tensors = batch["input_ids"].to('cuda')
#     attention_mask = batch["attention_mask"].to('cuda')

#     # Generate responses
#     response_tensors = ppo_trainer.model.policy.generate( # Call generate on ppo_trainer.model.policy
#         query_tensors,
#         attention_mask=attention_mask,
#         # REMOVED: max_length=max_seq_length,
#         max_new_tokens=64, # FIX: Use max_new_tokens to specify how many NEW tokens to generate.
#                            # Be aware: The total generated sequence length (input_length + max_new_tokens)
#                            # might exceed the max_seq_length (64) used by reward_tokenizer,
#                            # leading to truncation for reward calculation. Adjust this value
#                            # based on expected response length and desired truncation behavior.
#         do_sample=True,
#         top_k=50,
#         top_p=0.95,
#         num_return_sequences=1
#     )

#     # Prepare inputs for the Reward Model
#     queries = [tokenizer.decode(q, skip_special_tokens=True) for q in query_tensors]
#     responses = [tokenizer.decode(r, skip_special_tokens=True) for r in response_tensors]
    
#     # Extract the problem part from each query
#     reward_inputs = []
#     for q, r in zip(queries, responses):
#         # Split the query to isolate the problem part
#         problem_part = q.split('Problem: ')[1].split('\nSolution: ')[0] if 'Problem: ' in q and '\nSolution: ' in q else q
        
#         # Construct the reward input in the required format
#         reward_input = f"Problem: {problem_part}\nSolution: {r}"
#         reward_inputs.append(reward_input)

#     # Tokenize for reward model. Note: max_length=max_seq_length here still applies,
#     # so the combined 'Problem: ... Solution: ...' will be truncated to 64 tokens.
#     reward_tokenized = reward_tokenizer(reward_inputs, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt").to('cuda')

#     # Get rewards
#     with torch.no_grad():
#         reward_outputs = reward_model(**reward_tokenized)
#         rewards = reward_outputs.logits.squeeze()
#         if rewards.ndim == 0: # Ensure it's a 1-dim tensor if batch_size=1
#             rewards = rewards.unsqueeze(0)

#     # PPO step
#     stats = ppo_trainer.step(
#         query_tensors,
#         response_tensors,
#         rewards
#     )

#     if step % 10 == 0:
#         print(f"Step {step} - PPO stats: {stats}")

#     if step >= 50:  # Limit for demonstration; adjust as needed
#         print(f"Reached step {step}, stopping training.")
#         break

# # Save the RLHF-trained model
# rlhf_model_save_path = "/kaggle/working/outputs/rlhf_model"
# print(f"Saving RLHF-trained model to {rlhf_model_save_path}")
# model.save_pretrained(rlhf_model_save_path, torch_dtype=torch.float16)
# tokenizer.save_pretrained(rlhf_model_save_path)
# print("RLHF model saved. Contents:", os.listdir(rlhf_model_save_path))

# # Clear GPU memory after training
# print("Clearing GPU memory after RLHF training...")
# torch.cuda.empty_cache()
# del model
# del reward_model
# del ppo_trainer
# del rlhf_dataset
# gc.collect()
# for i in range(num_gpus):
#     print(f"GPU {i} memory allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
#     print(f"GPU {i} memory reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")

# # Final disk usage check
# total, used, free = shutil.disk_usage("/kaggle/working")
# print(f"Final disk usage - Total: {total / (1024**3):.2f} GB, Used: {used / (1024**3):.2f} GB, Free: {free / (1024**3):.2f} GB")

Number of available GPUs: 2
GPU 0 memory allocated: 13520.06 MB
GPU 0 memory reserved: 13784.00 MB
GPU 1 memory allocated: 1603.61 MB
GPU 1 memory reserved: 4712.00 MB
Initial disk usage - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB
Performing aggressive cleanup before starting RLHF...
Deleting model...
Deleting reward_model...
GPU 0 memory allocated after cleanup: 6020.44 MB
GPU 0 memory reserved after cleanup: 9244.00 MB
GPU 1 memory allocated after cleanup: 18.38 MB
GPU 1 memory reserved after cleanup: 1558.00 MB
Disk usage after cleanup - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB

⚡️ Starting Reinforcement Learning Fine-Tuning (RLHF)

Loading SFT-trained model in float16 from /kaggle/working/outputs/sft_model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model config - pad_token_id: 50257, eos_token_id: 50256
Loading trained Reward Model from /kaggle/working/outputs/qwen_reward_trained


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPU 0 memory allocated after loading models: 8917.66 MB
GPU 0 memory reserved: 9276.00 MB
GPU 1 memory allocated after loading models: 3086.35 MB
GPU 1 memory reserved: 3138.00 MB
Disk usage after loading models - Total: 19.52 GB, Used: 14.70 GB, Free: 4.80 GB


Map:   0%|          | 0/191 [00:00<?, ? examples/s]

PPOTrainer initialized.
Starting RLHF training with PPO...


TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <built-in method layer_norm of type object at 0x7b0379af6f80>(*(FakeTensor(..., device='cuda:1', size=(s0, s1, 1600), dtype=torch.float16), (1600,), Parameter(FakeTensor(..., device='cuda:0', size=(1600,), dtype=torch.float16,
           requires_grad=True)), Parameter(FakeTensor(..., device='cuda:0', size=(1600,), dtype=torch.float16,
           requires_grad=True)), 1e-05, True), **{}): got RuntimeError('Tensor on device cuda:0 is not on the expected device cuda:1!')

from user code:
   File "/usr/local/lib/python3.11/dist-packages/unsloth_zoo/patch_torch_functions.py", line 67, in layer_norm
    return torch.layer_norm(

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:
# Load reward model
print(f"Loading reward model from {reward_trained_model_dir}")
trained_reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_trained_model_dir,
    num_labels=1,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map=None
)
trained_reward_model = trained_reward_model.to('cuda:0')
if os.path.exists(os.path.join(reward_trained_model_dir, "adapter_model.safetensors")):
    trained_reward_model = PeftModelForSequenceClassification.from_pretrained(
        trained_reward_model,
        reward_trained_model_dir,
        torch_dtype=torch.float16,
        device_map=None
    )
    trained_reward_model = trained_reward_model.to('cuda:0')
for name, param in trained_reward_model.named_parameters():
    if "lora" in name.lower():
        param.data = param.data.to(torch.float16)
        print(f"Cast {name} to float16")
trained_reward_tokenizer = AutoTokenizer.from_pretrained(reward_trained_model_dir, trust_remote_code=True)
trained_reward_tokenizer.padding_side = 'left'
if trained_reward_tokenizer.pad_token is None or trained_reward_tokenizer.pad_token == trained_reward_tokenizer.eos_token:
    trained_reward_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
trained_reward_model.config.pad_token_id = trained_reward_tokenizer.convert_tokens_to_ids(trained_reward_tokenizer.pad_token)
print(f"Reward model pad_token_id: {trained_reward_model.config.pad_token_id}")

print(f"Policy model device: {next(model.parameters()).device}")
print(f"Reward model device: {next(trained_reward_model.parameters()).device}")

In [ ]:
def get_reward_score(problem_texts, response_texts, reward_model, reward_tokenizer, device):
    reward_model.eval()
    reward_model = reward_model.to(device)
    texts_for_rm = [f"Problem: {p}\nSolution: {r}" for p, r in zip(problem_texts, response_texts)]
    inputs = reward_tokenizer(
        texts_for_rm,
        padding=True,
        truncation=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = reward_model(**inputs)
        scores = outputs.logits.squeeze(-1)
    return scores.cpu()

ppo_args = TrainingArguments(
    output_dir=os.path.join(output_dir, "ppo_checkpoints"),
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    num_train_epochs=4,
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    report_to="none",
    fp16=True,
    bf16=False
)

ppo_config = PPOConfig(
    mini_batch_size=2,
    batch_size=8
)

max_ppo_epochs = 4

print(f"Loading reference model from {sft_model_save_path}")
ref_model = AutoModelForCausalLM.from_pretrained(
    sft_model_save_path,
    torch_dtype=torch.float16,
    device_map=None,
    attn_implementation="eager"
)
ref_model = ref_model.to('cuda:0')
if os.path.exists(os.path.join(sft_model_save_path, "adapter_model.safetensors")):
    ref_model = PeftModel.from_pretrained(ref_model, sft_model_save_path, torch_dtype=torch.float16, device_map=None)
    ref_model = ref_model.to('cuda:0')
for name, param in ref_model.named_parameters():
    if "lora" in name.lower():
        param.data = param.data.to(torch.float16)
        print(f"Cast {name} to float16")
    param.requires_grad = False
ref_model.eval()

print(f"Loading value model from {sft_model_save_path}")
value_model = AutoModelForCausalLM.from_pretrained(
    sft_model_save_path,
    torch_dtype=torch.float16,
    device_map=None,
    attn_implementation="eager"
)
value_model = value_model.to('cuda:0')
if os.path.exists(os.path.join(sft_model_save_path, "adapter_model.safetensors")):
    value_model = PeftModel.from_pretrained(value_model, sft_model_save_path, torch_dtype=torch.float16, device_map=None)
    value_model = value_model.to('cuda:0')
for name, param in value_model.named_parameters():
    if "lora" in name.lower():
        param.data = param.data.to(torch.float16)
        print(f"Cast {name} to float16")

print(f"PPOTrainer device: {ppo_trainer.accelerator.device}")
print(f"Policy model device: {next(model.parameters()).device}")
print(f"Reference model device: {next(ref_model.parameters()).device}")
print(f"Value model device: {next(value_model.parameters()).device}")
print(f"Reward model device: {next(trained_reward_model.parameters()).device}")

def tokenize_rlhf_dataset_for_ppo(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples["problem"],
        truncation=True,
        max_length=max_seq_length,
        padding='max_length',
        return_attention_mask=True
    )
    tokenized_inputs["problem"] = examples["problem"]
    return tokenized_inputs

tokenized_rlhf_dataset = rlhf_dataset.map(
    lambda x: tokenize_rlhf_dataset_for_ppo(x, tokenizer),
    batched=True,
    remove_columns=[col for col in rlhf_dataset.column_names if col not in ['input_ids', 'attention_mask', 'problem']]
)
print("✅ RLHF dataset tokenized for PPO.")

print("Setting generation parameters on model.generation_config...")
model.generation_config.max_new_tokens = max_seq_length
model.generation_config.eos_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id
model.generation_config.do_sample = True
model.generation_config.top_k = 50
model.generation_config.top_p = 0.95
print("Generation parameters configured.")

In [ ]:
class PPODatasetCollator(DataCollatorWithPadding):
    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        batch = {}
        tensor_features = [{k: v for k, v in feature.items() if k in ['input_ids', 'attention_mask']} for feature in features]
        padded_inputs = super().__call__(tensor_features)
        batch.update({
            'input_ids': padded_inputs['input_ids'].to(torch.int64),
            'attention_mask': padded_inputs['attention_mask'].to(torch.int64)
        })
        batch['problem'] = [feature['problem'] for feature in features]
        return batch

ppo_trainer = PPOTrainer(
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    train_dataset=tokenized_rlhf_dataset,
    args=ppo_args,
    reward_model=trained_reward_model,
    value_model=value_model,
    data_collator=PPODatasetCollator(tokenizer=tokenizer, padding=True, max_length=max_seq_length)
)

In [ ]:
print("Starting RLHF training with manual loop...")
rlhf_accuracies = []
rlhf_reasonings = []

for epoch in range(max_ppo_epochs):
    print(f"\n--- RLHF Epoch {epoch + 1}/{max_ppo_epochs} ---")
    for batch in ppo_trainer.dataloader:
        query_tensors = batch["input_ids"].to(ppo_trainer.accelerator.device, dtype=torch.int64)
        attention_mask = batch["attention_mask"].to(ppo_trainer.accelerator.device, dtype=torch.int64)
        problem_texts_batch = batch["problem"]

        print(f"query_tensors shape: {query_tensors.shape}, dtype: {query_tensors.dtype}")
        print(f"attention_mask shape: {attention_mask.shape}, dtype: {attention_mask.dtype}")
        print(f"attention_mask sample: {attention_mask[0]}")
        assert torch.all((attention_mask == 0) | (attention_mask == 1)), "Attention mask contains invalid values"

        response_tensors = model.generate(
            input_ids=query_tensors,
            attention_mask=attention_mask,
            max_new_tokens=max_seq_length,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
        response_texts = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

        rewards = get_reward_score(problem_texts_batch, response_texts, trained_reward_model, trained_reward_tokenizer, device=ppo_trainer.accelerator.device)
        rewards = rewards.detach()

        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

    print(f"\nEvaluating model after RLHF Epoch {epoch + 1}...")
    epoch_accuracy, epoch_reasoning = evaluate_model(reward_val, model, tokenizer)
    rlhf_accuracies.append(epoch_accuracy)
    rlhf_reasonings.append(epoch_reasoning)
    print(f"Epoch {epoch + 1} - Validation Accuracy: {epoch_accuracy:.2%}, Reasoning Quality: {epoch_reasoning:.2f}")

print("\n📊 Evaluating Final RLHF Model...")
try:
    final_accuracy, final_reasoning = evaluate_model(reward_val, model, tokenizer)
    print(f"\n✨ Final Evaluation - Accuracy: {final_accuracy:.2%}, Reasoning Quality: {final_reasoning:.2f}\n")
except Exception as e:
    print(f"Error during final evaluation: {str(e)}")
    raise

print(f"💾 Saving final PPO-trained policy model to {ppo_model_save_path}")
try:
    model.save_pretrained(ppo_model_save_path)
    tokenizer.save_pretrained(ppo_model_save_path)
    print("✅ Final PPO-trained policy model saved.")
except Exception as e:
    print(f"Error saving final PPO model: {str(e)}")

plt.figure(figsize=(10, 6))
plt.plot(range(1, max_ppo_epochs + 1), rlhf_accuracies, marker='o', label="RLHF Validation Accuracy")
plt.plot(range(1, max_ppo_epochs + 1), rlhf_reasonings, marker='x', label="RLHF Validation Reasoning Quality")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("RLHF Validation Metrics per Epoch")
plt.legend()
plt.savefig(os.path.join(output_dir, "rlhf_validation_metrics.png"))
plt.close()
print(f"RLHF validation metrics plot saved to {os.path.join(output_dir, 'rlhf_validation_metrics.png')}")